In [1]:
import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars org.apache.spark_spark-sql-kafka-0-10_2.11-2.4.6.jar pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.6 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import math
import string
import os
import random
from pyspark.sql import functions as F

In [3]:
import sys

In [4]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
#spark = SparkSession.builder.getOrCreate()

In [5]:
KAFKA_INPUT_TOPIC_NAME_CONS = 'quickstart-events'
KAFKA_OUTPUT_TOPIC_NAME_CONS = 'outputmallstream'
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'
MALL_LONGITUDE=78.446841
MALL_LATITUDE=17.427229
MALL_THRESHOLD_DISTANCE=100

In [6]:
spark = SparkSession \
.builder \
.appName('PySpark Structured Streaming with Kafka') \
.master('local[*]') \
.getOrCreate()

In [ ]:
#import pyspark

In [ ]:
#conf = pyspark.SparkConf().setAll([('spark.executor.memory', '2g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','2g')])
#spark.stop()
#sc = spark.sparkContext
#sc.stop()
#spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [7]:
stream_detail_df = spark.readStream.format('kafka').option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS_CONS).option('subscribe', KAFKA_INPUT_TOPIC_NAME_CONS) \
.option('startingOffsets', 'latest').load()

In [8]:
stream_detail_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
stream_detail_df = stream_detail_df.selectExpr('CAST(key AS STRING)','CAST(value AS STRING)', 'timestamp')

In [ ]:
stream_detail_df.writeStream \
      .format("parquet") \
      .outputMode("append") \
      .option("path","c:/tmp/spark_out/parquet") \
      .option("checkpointLocation", "c:/tmp/checkpoint") \
      .trigger(processingTime='60 seconds')\
      .start() \
      .awaitTermination()

In [ ]:
## https://cumsum.wordpress.com/2019/02/06/typeerror-invalid-argument-not-a-string-or-column-mysterious-error-when-creating-a-udf/

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia=SIA()

In [7]:
def sentiment_detection(text):
    #text1= tokenizer.tokenize(text)
    #
    score = sia.polarity_scores(text)
    score=score['compound']
    if score>=0.05:
        sentiment='Positive'
    elif score<=(-0.05):
        sentiment='Negative'
    else:
        sentiment='Neutral'
        
    ##total operation now will be here
    #sentiment='maoo'
    return sentiment

In [8]:
def text_classification(words):
    sentiment_detection_udf = udf(sentiment_detection, StringType())
    words = words.withColumn("sentiment", sentiment_detection_udf("value")) ##column name here will pass 
    return words


In [9]:
if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName('PySpark Structured Streaming with Kafka') \
        .master('local[*]') \
        .getOrCreate()
    stream_detail_df = spark.readStream.format('kafka').option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS_CONS).option('subscribe', KAFKA_INPUT_TOPIC_NAME_CONS) \
        .option('startingOffsets', 'latest').load()
    stream_detail_df = stream_detail_df.selectExpr('CAST(key AS STRING)','CAST(value AS STRING)','timestamp')
    
    words = text_classification(stream_detail_df)

    query=words.writeStream \
      .format("parquet") \
      .outputMode("append") \
      .option("path","c:/tmp/spark_out/parquet") \
      .option("checkpointLocation", "c:/tmp/checkpoint") \
      .start() 
    query.awaitTermination()  ##optional value 2000

In [12]:
stream_detail_df = stream_detail_df.selectExpr('CAST(key AS STRING)','CAST(value AS STRING)','timestamp')
    
words = text_classification(stream_detail_df)

In [13]:
customer_detail_write_stream_1 = words.writeStream \
.format('kafka') \
.option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS_CONS) \
.option('topic', KAFKA_OUTPUT_TOPIC_NAME_CONS) \
.trigger(processingTime='60 seconds') \
.outputMode('update') \
.option('checkpointLocation','C:\pyspark_testing\checkpoint') \
.start() \
.awaitTermination()  
